### Step 1: Import model

In [37]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, DenseFeat,get_feature_names
import numpy as np

data = pd.read_csv('./criteo_sample.txt')

sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I'+str(i) for i in range(1, 14)]

data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0,)
target = ['label']

In [38]:
print(sparse_features)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26']


In [39]:
print(dense_features)

['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12', 'I13']


In [40]:
print(np.shape(data))
print(data[0:10])

(200, 40)
   label    I1   I2     I3    I4       I5     I6    I7    I8     I9  ...  \
0      0   0.0    3  260.0   0.0  17668.0    0.0   0.0  33.0    0.0  ...   
1      0   0.0   -1   19.0  35.0  30251.0  247.0   1.0  35.0  160.0  ...   
2      0   0.0    0    2.0  12.0   2013.0  164.0   6.0  35.0  523.0  ...   
3      0   0.0   13    1.0   4.0  16836.0  200.0   5.0   4.0   29.0  ...   
4      0   0.0    0  104.0  27.0   1990.0  142.0   4.0  32.0   37.0  ...   
5      0   0.0   -1   63.0  40.0   1470.0   61.0   4.0  37.0   46.0  ...   
6      0   0.0  370    4.0   1.0   1787.0   65.0  14.0  25.0  489.0  ...   
7      1  19.0   10   30.0  10.0      1.0    3.0  33.0  47.0  126.0  ...   
8      0   0.0    0   36.0  22.0   4684.0  217.0   9.0  35.0  135.0  ...   
9      0   2.0   11    8.0  23.0     30.0   11.0   2.0   8.0   23.0  ...   

        C17       C18       C19       C20       C21 C22       C23       C24  \
0  e5ba7672  87c6f83c        -1        -1  0429f84b  -1  3a171ecb  c0d61a5

### Step 2: Simple preprocessing
**Label Encoding: map the features to integer value from 0 ~ len(#unique) - 1**

In [41]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

**Hash Encoding: map the features to a fix range,like 0 ~ 9999.We have 2 methods to do that:**

Do feature hashing before training

In [16]:
for feat in sparse_features:
    lbe = HashEncoder()
    data[feat] = lbe.transform(data[feat])

NameError: name 'HashEncoder' is not defined

Do feature hashing on the fly in training process

We can do feature hashing by setting use_hash=True in SparseFeat or VarlenSparseFeat in Step3.

**And for dense numerical features,they are usually discretized to buckets,here we use normalization.**

In [42]:
mms = MinMaxScaler(feature_range=(0,1))
data[dense_features] = mms.fit_transform(data[dense_features])

### Step 3: Generate feature columns

For sparse features, we transform them into dense vectors by embedding techniques. For dense numerical features, we concatenate them to the input tensors of fully connected layer.

And for varlen(multi-valued) sparse features,you can use VarlenSparseFeat. Visit examples of using VarlenSparseFeat

**Label Encoding**